In [83]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = [16,9]
%matplotlib inline

In [84]:
#Parsing date column of both datasets to be read by python

pdata = pd.read_csv('data/pollution-1.csv',parse_dates=['ReadingDateTime'])
wdata = pd.read_csv('data/weather-1.csv',parse_dates=['DATE'])

In [85]:
#Extracting numerical values of dew point, temperature, wind speed (converting from mps to kmph) and direction

wdata['DEW'] = wdata['DEW'].str[:-2].astype(np.float64)/10
wdata['TMP'] = wdata['TMP'].str[:-2].astype(np.float64)/10
wdata['DIR'] = wdata['WND'].str[:3].astype(np.float64)
wdata['SPD'] = (wdata['WND'].str[8:-2].astype(np.float64)/10)*3.6

#Calculating relative humidity from dew point and temperature

wdata['HUM'] = 100*(np.exp((17.625 * wdata['DEW'])/(243.04 + wdata['DEW']))/np.exp((17.625 * wdata['TMP'])/(243.04 + wdata['TMP'])))

#Replacing missing wind direction observations with nulls

wdata.DIR.replace(999,np.nan,inplace=True)

In [72]:
#Downsampling the weather dataset

wdata.index=wdata['DATE']
wdata_resampled = wdata.resample('1H').mean()[:17520]
wdata = wdata_resampled.reset_index()

In [73]:
#Joining the weather and pollution datasets

data = pd.concat([wdata['DATE'], wdata['DEW'],wdata['TMP'],wdata['DIR'],wdata['SPD'],wdata['HUM'],pdata.Value[pdata.Species=='NO2'].reset_index(drop=True).rename('NO2'),pdata.Value[(pdata.Species=='O3')].reset_index(drop=True).rename('O3'),pdata.Value[(pdata.Species=='PM2.5')].reset_index(drop=True).rename('PM25')], axis=1)

In [74]:
#Number of possible outilers

print('DEW:',((data['DEW'] <= -99) | (data['DEW'] >= 37)).sum())

print('TMP:',((data['TMP'] <= -94) | (data['TMP'] >= 62)).sum())

print('DIR:',((data['DIR'] < 1) | (data['DIR'] > 360)).sum())

print('SPD:',((data['SPD'] < 0) | (data['SPD'] > 324)).sum())

print('HUM:',((data['HUM'] < 0) | (data['HUM'] > 100)).sum())

print('NO2:',((data['NO2'] < 0) | (data['NO2'] > 601)).sum())

print('O3:',((data['O3'] < 0) | (data['O3'] > 241)).sum())

print('PM25:',((data['PM25'] < 0) | (data['PM25'] > 71)).sum())

DEW: 9
TMP: 9
DIR: 0
SPD: 10
HUM: 0
NO2: 0
O3: 9
PM25: 128


In [75]:
#Max and min values before removing outliers

minmax = {'Species': ['DEW', 'TMP', 'DIR', 'SPD', 'HUM','NO2','O3','PM25'], 
        'Min': [data['DEW'].min(), data['TMP'].min(),data['DIR'].min(),data['SPD'].min(),data['HUM'].min(),data['NO2'].min(),data['O3'].min(),data['PM25'].min()], 
        'Max': [data['DEW'].max(), data['TMP'].max(),data['DIR'].max(),data['SPD'].max(),data['HUM'].max(),data['NO2'].max(),data['O3'].max(),data['PM25'].max()]}
pd.DataFrame(minmax, columns = ['Species', 'Min', 'Max'])

,Species,Min,Max
0,DEW,-9.000000,508.95000
1,TMP,-5.000000,513.45000
2,DIR,10.000000,360.00000
3,SPD,0.000000,3599.64000
4,HUM,21.789608,100.00000
5,NO2,4.300000,181.39999
6,O3,-1.600000,130.70000
7,PM25,-5.800000,101.20000


In [86]:
#Replacing outliers with nulls
#Ozone cannot be negative
#PM2.5 outlier is a possible value
dew_out = data.DEW[(data['DEW'] <= -99) | (data['DEW'] >= 37)]
tmp_out = data.TMP[(data['TMP'] <= -94) | (data['TMP'] >= 62)]
spd_out = data.SPD[(data['SPD'] < 0) | (data['SPD'] > 324)]
o3_out = data.O3[(data['O3'] < 0) | (data['O3'] > 241)]
pm25_out = data.PM25[(data['PM25'] < 0)]
data.DEW.replace(dew_out,np.nan,inplace=True)
data.TMP.replace(tmp_out,np.nan,inplace=True)
data.SPD.replace(spd_out,np.nan,inplace=True)
data.O3.replace(o3_out,np.nan,inplace=True)
data.PM25.replace(pm25_out,np.nan,inplace=True)

In [87]:
# Add code here to fill null values
# data.to_csv('fillna.csv',index=False)

In [88]:
#Checking number of null values

# data.isna().sum()
#data_mean = pd.read_csv('fillnamean.csv',parse_dates=['DATE'])
#data_recent = pd.read_csv('fillnalatest.csv',parse_dates=['DATE'])

In [89]:
#Adding date features

data['MONTH'] = pd.DatetimeIndex(data['DATE']).month
data['HOUR'] = pd.DatetimeIndex(data['DATE']).hour
data['DAY'] = data['DATE'].dt.weekday_name
data['DAY_CAT'] = data.DAY.astype("category").cat.codes

In [90]:
# Adding time shifted features
## This is done before removing nulls so that the dataset is continuous
### To be cleaned up into a function

data['DEW_1'] = data.DEW.shift(periods=1)
data['TMP_1'] = data.TMP.shift(periods=1)
data['DIR_1'] = data.DIR.shift(periods=1)
data['SPD_1'] = data.SPD.shift(periods=1)
data['HUM_1'] = data.HUM.shift(periods=1)
data['NO2_1'] = data.NO2.shift(periods=1)
data['O3_1'] = data.O3.shift(periods=1)
data['PM25_1'] = data.PM25.shift(periods=1)

data['DEW_2'] = data.DEW.shift(periods=2)
data['TMP_2'] = data.TMP.shift(periods=2)
data['DIR_2'] = data.DIR.shift(periods=2)
data['HUM_2'] = data.HUM.shift(periods=2)
data['SPD_2'] = data.SPD.shift(periods=2)
data['NO2_2'] = data.NO2.shift(periods=2)
data['O3_2'] = data.O3.shift(periods=2)
data['PM25_2'] = data.PM25.shift(periods=2)

data['DEW_3'] = data.DEW.shift(periods=3)
data['TMP_3'] = data.TMP.shift(periods=3)
data['DIR_3'] = data.DIR.shift(periods=3)
data['HUM_3'] = data.HUM.shift(periods=3)
data['SPD_3'] = data.SPD.shift(periods=3)
data['NO2_3'] = data.NO2.shift(periods=3)
data['O3_3'] = data.O3.shift(periods=3)
data['PM25_3'] = data.PM25.shift(periods=3)

data['DEW_4'] = data.DEW.shift(periods=4)
data['TMP_4'] = data.TMP.shift(periods=4)
data['DIR_4'] = data.DIR.shift(periods=4)
data['HUM_4'] = data.HUM.shift(periods=4)
data['SPD_4'] = data.SPD.shift(periods=4)
data['NO2_4'] = data.NO2.shift(periods=4)
data['O3_4'] = data.O3.shift(periods=4)
data['PM25_4'] = data.PM25.shift(periods=4)

data['DEW_5'] = data.DEW.shift(periods=5)
data['TMP_5'] = data.TMP.shift(periods=5)
data['DIR_5'] = data.DIR.shift(periods=5)
data['HUM_5'] = data.HUM.shift(periods=5)
data['SPD_5'] = data.SPD.shift(periods=5)
data['NO2_5'] = data.NO2.shift(periods=5)
data['O3_5'] = data.O3.shift(periods=5)
data['PM25_5'] = data.PM25.shift(periods=5)

In [ ]:
#Checking number of null values

data.isna().sum()

In [ ]:
#Percentage of null values in final dataset

data.isna().mean().round(4) * 100

In [91]:
# Show rows where any cell has a NaN
data[data.isnull().any(axis=1)].shape

(6314, 53)

In [68]:
# data_final = data.dropna(axis=0).reset_index(drop=True)
# data_final = data_final.drop(['DATE','DAY'], axis=1)
# data_final.to_csv('final.csv',index=False)

In [82]:
# data_noshift = data.dropna(axis=0).reset_index(drop=True)
# data_noshift = data_noshift.drop(['DATE','DAY'], axis=1)
# data_noshift.to_csv('final-noshift.csv',index=False)

In [82]:
# data_new = data.dropna(axis=0).reset_index(drop=True)
# data_new.drop(['DATE','DAY'], axis=1,inplace=True)
# data_new.to_csv('final-new.csv',index=False)

In [53]:
# data_mean = data.dropna(axis=0).reset_index(drop=True)
# data_mean.drop(['DATE','DAY'], axis=1,inplace=True)
# data_mean.to_csv('final-mean.csv',index=False)

In [67]:
# data_recent = data.dropna(axis=0).reset_index(drop=True)
# data_recent.drop(['DATE','DAY'], axis=1,inplace=True)
# data_recent.to_csv('final-recent.csv',index=False)

In [ ]:
# data_filled = data.fillna(value=0)
# data_filled = data_filled.drop(['DATE','DAY'], axis=1)
# data_filled.to_csv('final-filled.csv',index=False)

In [92]:
#  data.to_csv('final-visualization.csv',index=False)